In [ ]:
#r "nuget: MathNet.Numerics.FSharp, 4.15.0"

Installed package MathNet.Numerics.FSharp version 4.15.0

In [ ]:
open MathNet.Numerics.LinearAlgebra
open MathNet.Numerics.Integration
open System.Collections.Generic

Рассмотрим краевую задачу для линейного обыкновенного дифференциального уравнения
второго порядка с однородными граничными условиями.

In [ ]:
let l u du d2u = fun x ->
    - (6. + x) / (7. + 3. * x) * d2u x
    - (1. - x / 2.) * du x
    + (1. + 0.5 * cos x) * u x

let f = fun x -> 1. - x / 3.

let left = -1.
let right = 1.

let (alpha1, alpha2) = -2., -1
let (beta1, beta2) = 0., 1

In [ ]:
let k = 2

/// k - верхний индекс, n - максимальная степень
let jacobiPolynomials k =
    let rec recurrenceRelation n prevPoly2 prevPoly = seq {
        yield prevPoly2
        yield! 
            fun (x: float) -> 
                (
                    (float n + float k + 2.) * (2. * float n + 2. * float k + 3.) * x * prevPoly x - 
                    (float n + float k + 2.) * (float n + float k + 1.) * prevPoly2 x
                ) 
                / ((float n + 2. * float k + 2.) * (float n + 2.))
            |> recurrenceRelation (n + 1) prevPoly
    }   

    recurrenceRelation 0 (fun x -> 1.) (fun x -> (float k + 1.) * x) 

let jacobiDerivatives k =
    let jacobi = jacobiPolynomials (k + 1)

    let rec recurrenceRelation n = seq {
        yield fun (x: float) -> (float n + 2. * float k + 1.) / 2. * (jacobi |> Seq.item (n - 1)) x
        yield! recurrenceRelation (n + 1)
    }   

    seq {
        yield (fun x -> 0.) 
        yield! recurrenceRelation 1
    }
   
let jacobiDerivatives2 k = 
    let jacobi = jacobiPolynomials (k + 2)

    let rec recurrenceRelation n = seq {
        yield fun (x: float) -> (float n + 2. * float k + 1.) * (float n + 2. * float k + 2.) / 4. * (jacobi |> Seq.item (n - 2)) x
        yield! recurrenceRelation (n + 1)
    }   

    seq {
        yield (fun x -> 0.) 
        yield (fun x -> 0.) 
        yield! recurrenceRelation 2  
    } 

In [ ]:
let w1 = fun x -> x ** 2. - 2. * x - 5.
let dw1 = fun x -> 2. * x - 2.
let ddw1 = fun x -> 2.

let w2 = fun x -> x ** 3. - 3. * x + 2.
let dw2 = fun x -> 3. * x ** 2. - 3.
let ddw2 = fun x -> 6. * x

In [ ]:
let coordinateFunctionsAndDerivatives =
    let jacobi = jacobiPolynomials k
    let jacobiD = jacobiDerivatives k
    let jacobiD2 = jacobiDerivatives2 k

    let rec recurrenceRelation n = seq {
        yield 
            (fun (x: float) -> (1. - x ** 2.) ** 2. * (Seq.item (n - 2) jacobi) x),
            (fun (x: float) -> -2. * float (n - 2) * (1. - x ** 2.) * (Seq.item (n - 1) jacobi) x),
            (fun (x: float) -> -2. * float (n - 2) * (((-2. * x) * (Seq.item (n - 1) jacobi) x + (1. - x ** 2.) * (float (n - 2) + 3.)) / 2.) * (Seq.item (n - 2) jacobi) x)
            
        yield! recurrenceRelation (n + 1)
    } 

    seq {
        yield (w1, dw1, ddw1)
        yield (w2, dw2, ddw2)
        yield! recurrenceRelation 2
    }

In [ ]:
let n = 7
let coordinates = 
    coordinateFunctionsAndDerivatives 
    |> Seq.take n
    |> Seq.toList

let a = DenseMatrix.init n n (fun i j ->
    let (wi, dwi, ddwi) = coordinates.[i]
    let (wj, dwj, ddwj) = coordinates.[j]
    let lwj = l wj dwj ddwj
    let lwi = l wi dwi ddwi
    GaussLegendreRule.Integrate((fun x -> lwj x * lwi x), left, right, 20)
)    

let b = DenseVector.init n (fun i ->
    let (wi, dwi, ddwi) = coordinates.[i]
    let lwi = l wi dwi ddwi
    GaussLegendreRule.Integrate((fun x -> f x * lwi x), left, right, 20)
)

type Info = {
    N: int
    CondA: float
    Yn_05: float
    Yn0: float
    Yn05: float
}

let table = List<Info>()

for k = 3 to n do
    let c = a.[0 .. k - 1, 0 .. k - 1].Solve(b.[0 .. k - 1])
    let u = fun x ->
        [ 0 .. k - 1 ]
        |> List.map 
            (fun i -> 
                let (wi, _, _) = coordinates.[i]
                wi x * c.[i]
            )
        |> List.sum
    
    {
        N = k
        CondA = a.[0 .. k - 1, 0 .. k - 1].ConditionNumber()
        Yn_05 = u -0.5
        Yn0 = u 0.
        Yn05 = u 0.5
    }
    |> table.Add

table

index,N,CondA,Yn_05,Yn0,Yn05
0,3,417.6371330289169,0.5969500889993333,0.4716895571601008,0.5557339880242801
1,4,1991.0724493971406,0.588851728499239,0.49885861995149716,0.5480519853642027
2,5,66079.255567005,0.5919749040715132,0.4763525852639595,0.5531347916774096
3,6,890110.6719833454,0.5905999247778897,0.4646259787127684,0.5518730735793704
4,7,17639724.79134643,0.5909121420363972,0.4558703417092177,0.5515554229563946


In [ ]:
let getChebyshevRoots n = 
    [0 .. (n - 1)]
    |> List.map (fun k -> Math.Cos (Math.PI * (float k + 0.5) / float n))

In [ ]:
let roots = getChebyshevRoots n

let aCollocation = DenseMatrix.init n n (fun i j ->
    let (wj, dwj, ddwj) = coordinates.[j]
    let lwj = l wj dwj ddwj
    lwj roots.[i]
)    

let bCollocation = DenseVector.init n (fun i ->
    f roots.[i]
)

let tableCollocation = List<Info>()

for k = 3 to n do
    let cCollocation = aCollocation.[0 .. k - 1, 0 .. k - 1].Solve(bCollocation.[0 .. k - 1])
    let uCollocation = fun x ->
        [ 0 .. k - 1 ]
        |> List.map 
            (fun i -> 
                let (wi, _, _) = coordinates.[i]
                wi x * cCollocation.[i]
            )
        |> List.sum
    
    {
        N = k
        CondA = aCollocation.[0 .. k - 1, 0 .. k - 1].ConditionNumber()
        Yn_05 = uCollocation -0.5
        Yn0 = uCollocation 0.
        Yn05 = uCollocation 0.5
    }
    |> tableCollocation.Add

tableCollocation

index,N,CondA,Yn_05,Yn0,Yn05
0,3,198.55566467364758,0.43049305815253897,0.5537852759352879,0.545079257226006
1,4,201.27274263047917,0.5646642035920786,0.5349428245974295,0.5606379924701927
2,5,405.28738574716874,0.5842656610150162,0.5102673627356759,0.5594621824606794
3,6,1026.8108642849083,0.5883734909029307,0.46465324867256486,0.5497129608472219
4,7,7557.66107257316,0.5885859491770903,0.4618169913395334,0.5494710826787325


In [ ]:
printfn "%A" <| (table.[0].Yn_05 - tableCollocation.[0].Yn_05, table.[4].Yn_05 - tableCollocation.[4].Yn_05)
printfn "%A" <| (table.[0].Yn0 - tableCollocation.[0].Yn0, table.[4].Yn0 - tableCollocation.[4].Yn0)
printfn "%A" <| (table.[0].Yn05 - tableCollocation.[0].Yn05, table.[4].Yn05 - tableCollocation.[4].Yn05)


(0.1664570308, 0.002326192859)

(-0.08209571878, -0.00594664963)

(0.0106547308, 0.002084340278)